In [7]:
import pandas as pd
from scipy import sparse
import pandas as pd
import numpy as np
from numpy import var
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import random
from matplotlib import pyplot as plt
from scipy.stats import norm
import numpy as np
from fitter import Fitter, get_common_distributions, get_distributions
from scipy import stats
from scipy.stats import norm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

#load data
ratings = pd.read_csv('dataset/ratings.csv')
moviesPartial = pd.read_csv('dataset/movies.csv')
moviesFull = pd.read_csv('dataset/movie_dataset.csv')

#drop duplicates
moviesFull.drop_duplicates(subset = 'title', inplace=True)

#drop columns that do nothing
ratings = ratings.drop(['timestamp'], axis = 1)
moviesPartial = moviesPartial.drop(['genres'],axis=1)

#all the users are to be tested
userRatings = []
for i in range(1,611):
    userRatings.append(ratings[ratings['userId'] ==i])

#remove the year and keep the name
moviesPartial['title'] = moviesPartial['title'].map(lambda string: string[:-7])

#drop duplicates
moviesPartial.drop_duplicates(subset = 'title', inplace=True)

#helper functions:
def get_title_from_movieId(tmp):
    return moviesPartial[moviesPartial["movieId"]== tmp]["title"].to_string(index=False)

def checkTitle(tmp):
    if(tmp not in list(moviesFull["title"])):
        #The word "the" is being apended to the ends of titles
        tmp = tmp[-3:] +" "+ tmp[:-5]
        return moviesFull[moviesFull["title"] == tmp]["title"].to_string(index=False)
    else:
        return moviesFull[moviesFull["title"] == tmp]["title"].to_string(index=False)



#used to filter out movies that are not in both data sets
newUserTitles = []
newUserRatings = []
for userRating in userRatings:
    temp1 = []
    temp2 = []
    for rating, id in zip(list(userRating["rating"]), list(userRating["movieId"])):
        title = get_title_from_movieId(id)
        checked = checkTitle(title)
        if(checked != 'Series([], )'):
            temp1.append(checked)
            temp2.append(rating)
    newUserTitles.append(temp1)
    newUserRatings.append(temp2)
    


#this might lead to a referencce error
#populate infos; a list of data frames
infos = []
for user in newUserTitles:
    i = 0
    tempInfo = pd.DataFrame(columns = list(moviesFull.columns))
    for name in user:
        if(len(moviesFull[moviesFull["title"] == name]) !=0):
            tempInfo.loc[i] = moviesFull[moviesFull["title"] == name].iloc[0]
        i+=1
    infos.append(tempInfo)
    

features = ['keywords','cast','genres','director']


for info in infos:
    for feature in features:
        info[feature] = info[feature].fillna('')

colNumbers = []
for feature in features:   
    colNumbers.append(infos[0].columns.get_loc(feature))

fullDataList =[]


for frame in infos:
    fullDataList.append(frame.values.tolist())

combinedFeatures = []

for item in fullDataList:
    tmp = []
    for row in item:
        tmp.append(row[colNumbers[0]]+" "+row[colNumbers[1]]+" "+row[colNumbers[2]]+" "+row[colNumbers[3]])
    combinedFeatures.append(tmp)


def transform1(arr):
    ret = []
    for item in arr:
        if(item<=.5):
            ret.append(1/item)
        if(item>.5):
            ret.append(-1*(1/(1-item)))
    return ret

def transform2(arr):
    ret = []
    for item in arr:
        ret.append(1/item)
    return ret

X = []
y = []
pred  = []

for i in range(len(combinedFeatures)):
    cv = CountVectorizer() 
    count_matrix = cv.fit_transform(combinedFeatures[i]).toarray()
    cosine_sim = cosine_similarity(X = count_matrix[1:count_matrix.shape[1]]
    ,Y = [count_matrix[0]])

    average = sum(newUserRatings[i][1:])/(len(newUserRatings[i])-1)
    ratings = newUserRatings[i][1:]
    similairities = np.reshape(cosine_sim,  (len(cosine_sim)))

    scores = stats.zscore(ratings)
    p_values = norm.sf(scores)
    vals1 = transform1(p_values)
    scores = stats.zscore(similairities)
    p_values = norm.sf(scores)
    vals2 = transform2(p_values)

    totalRating = 0

    s1 = abs(sum(vals1))
    s2 = abs(sum(vals2))

    for val1, val2,sim, rating in zip(vals1, vals2, similairities, ratings):
        totalRating += (val1/s1)*(val2/s2)
        

    print("calulated:", average+totalRating)
    print("actual:", newUserRatings[i][0])
    pred.append(average+totalRating)
    #next try just total rating
    X.append([average,totalRating])
    y.append(newUserRatings[i][0])


print("before linear reg", r2_score(pred, y))
#reshapedX= np.reshape(X, (len(X), 1))

reg = LinearRegression().fit(X, y)

#note: uses training data to test
print("after linear regression:", reg.score(X, y))

predicted = reg.predict(X)

for item, actual in zip(predicted, y):
    print("predicted:", item)
    print("actual:", actual)









calulated: 4.363602624449066
actual: 4.0
calulated: 4.042461408081289
actual: 3.0
calulated: 1.6907327644192498
actual: 0.5
calulated: 3.4343639947134403
actual: 3.0
calulated: 3.5758922696595814
actual: 4.0
calulated: 3.6501688076733805
actual: 3.0
calulated: 3.1243879709595146
actual: 4.5
calulated: 5.2206044476509135
actual: 2.0
calulated: 3.3786682310925324
actual: 3.0
calulated: 3.2488193522985784
actual: 1.0
calulated: 3.8178214908672334
actual: 3.0
calulated: 4.482904571573478
actual: 4.0
calulated: 3.7653232573352877
actual: 3.0
calulated: 2.9513342267309426
actual: 1.0
calulated: 3.3932975337145885
actual: 2.5
calulated: 3.649055225620497
actual: 4.0
calulated: 4.163929937980895
actual: 4.5
calulated: 3.6916170818281078
actual: 3.5
calulated: 2.6544187545658877
actual: 4.0
calulated: 3.482402033996301
actual: 4.0
calulated: 3.3892653785137172
actual: 3.5
calulated: 2.240133934701713
actual: 5.0
calulated: 3.4796564849401315
actual: 4.0
calulated: 3.6181101117953722
actual: 4.0